# Fake News Detection with Web Search Integration

This notebook provides a complete implementation of a fake news detection system that uses web search capabilities to verify claims. The system can work independently or be integrated with a CLIP-based model for enhanced detection.

## Overview

The system works by:
1. Extracting claims from news text
2. Checking these claims against fact-checking sources
3. Determining the credibility of the news based on fact-check results
4. (Optional) Integrating with a CLIP model for multimodal analysis

## 1. Setup and Installation

First, let's install the required packages:

In [ ]:
# Install required packages
!pip install nltk requests pandas

# Optional: Install PyTorch and transformers if you want to use the CLIP model integration
# !pip install torch transformers

# Download NLTK resources
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Import other required libraries
import os
import re
import json
import requests
import logging
from typing import List, Dict, Any, Optional, Tuple

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

## 2. API Key Setup

Enter your Google Fact Check API key below:

In [ ]:
# Set your API key here
API_KEY = input("Enter your Google Fact Check API key: ")

# Verify API key is provided
if not API_KEY:
    print("Warning: No API key provided. The system will not be able to perform fact-checking.")
else:
    print("API key set successfully!")
    
# Set as environment variable (optional)
os.environ["FACT_CHECK_API_KEY"] = API_KEY

## 3. Claim Extractor Implementation

This component extracts potential claims from news text for fact-checking.

In [ ]:
class ClaimExtractor:
    """
    Module for extracting claims from news text for fact-checking.
    """
    
    def __init__(self):
        """Initialize the ClaimExtractor."""
        # Claim indicator phrases
        self.claim_indicators = [
            "claims that", "stated that", "says that", "according to",
            "reported that", "announced that", "declared that", "asserted that",
            "confirmed that", "denied that", "suggested that", "alleged that",
            "revealed that", "mentioned that", "indicated that", "argued that"
        ]
        
        # Patterns for direct quotes
        self.quote_pattern = re.compile(r'"([^"]*)"')
        
    def extract_claims(self, text: str) -> List[str]:
        """
        Extract potential claims from text.
        
        Args:
            text: The news text to analyze
            
        Returns:
            List of extracted claim strings
        """
        if not text:
            logger.warning("Empty text provided for claim extraction")
            return []
        
        # Tokenize text into sentences
        sentences = nltk.sent_tokenize(text)
        
        # Extract claims using multiple methods
        claims = []
        claims.extend(self._extract_indicator_claims(sentences))
        claims.extend(self._extract_quote_claims(text))
        claims.extend(self._extract_statement_claims(sentences))
        
        # Remove duplicates and very short claims
        unique_claims = []
        for claim in claims:
            claim = claim.strip()
            if claim and len(claim) > 15 and claim not in unique_claims:
                unique_claims.append(claim)
        
        logger.info(f"Extracted {len(unique_claims)} unique claims from text")
        return unique_claims
    
    def _extract_indicator_claims(self, sentences: List[str]) -> List[str]:
        """
        Extract claims based on indicator phrases.
        
        Args:
            sentences: List of sentences from the text
            
        Returns:
            List of claims extracted using indicator phrases
        """
        claims = []
        
        for sentence in sentences:
            for indicator in self.claim_indicators:
                if indicator in sentence.lower():
                    # Split by the indicator and take the part after it
                    parts = sentence.lower().split(indicator, 1)
                    if len(parts) > 1:
                        claim = parts[1].strip()
                        # Clean up the claim
                        claim = re.sub(r'^\s*that\s+', '', claim)
                        claims.append(claim)
        
        return claims
    
    def _extract_quote_claims(self, text: str) -> List[str]:
        """
        Extract claims from quoted text.
        
        Args:
            text: The full text
            
        Returns:
            List of claims extracted from quotes
        """
        # Find all quoted text
        quotes = self.quote_pattern.findall(text)
        
        # Filter out short quotes or those that don't look like claims
        claims = []
        for quote in quotes:
            if len(quote) > 15 and any(word in quote.lower() for word in ["is", "are", "was", "were", "will", "has", "have"]):
                claims.append(quote)
        
        return claims
    
    def _extract_statement_claims(self, sentences: List[str]) -> List[str]:
        """
        Extract claims that appear to be statements of fact.
        
        Args:
            sentences: List of sentences from the text
            
        Returns:
            List of claims that appear to be statements of fact
        """
        claims = []
        
        for sentence in sentences:
            # Skip very short sentences
            if len(sentence) < 20:
                continue
                
            # POS tagging to identify sentence structure
            tagged = nltk.pos_tag(nltk.word_tokenize(sentence))
            
            # Check if the sentence starts with a proper noun or pronoun followed by a verb
            if len(tagged) > 2:
                first_tag = tagged[0][1]
                second_tag = tagged[1][1]
                
                # Check for patterns like "NNP VBZ" (e.g., "Trump says") or "PRP VBZ" (e.g., "He claims")
                if (first_tag.startswith('NNP') and second_tag.startswith('VB')) or \
                   (first_tag.startswith('PRP') and second_tag.startswith('VB')):
                    claims.append(sentence)
                
                # Check for sentences with factual verbs
                factual_verbs = ["is", "are", "was", "were", "will", "has", "have", "shows", "reveals", "confirms", "proves"]
                if any(word.lower() in factual_verbs for word, tag in tagged):
                    claims.append(sentence)
        
        return claims
    
    def rank_claims(self, claims: List[str], top_n: int = 3) -> List[str]:
        """
        Rank claims by importance and return the top N.
        
        Args:
            claims: List of extracted claims
            top_n: Number of top claims to return
            
        Returns:
            List of top N ranked claims
        """
        if not claims:
            return []
        
        # Simple ranking based on length and presence of key terms
        ranked_claims = []
        for claim in claims:
            score = 0
            
            # Longer claims might be more substantial
            score += min(len(claim) / 20, 3)
            
            # Claims with numbers might be more verifiable
            if re.search(r'\d+', claim):
                score += 2
                
            # Claims with named entities might be more important
            if re.search(r'[A-Z][a-z]+(?:\s+[A-Z][a-z]+)+', claim):
                score += 2
                
            # Claims with certain keywords might be more important
            importance_keywords = ["percent", "study", "research", "found", "according", "evidence", "data", "report"]
            for keyword in importance_keywords:
                if keyword in claim.lower():
                    score += 1
            
            ranked_claims.append((claim, score))
        
        # Sort by score in descending order
        ranked_claims.sort(key=lambda x: x[1], reverse=True)
        
        # Return top N claims
        return [claim for claim, score in ranked_claims[:top_n]]

## 4. Web Search Module Implementation

This component handles the interaction with the Google Fact Check API.

In [ ]:
class WebSearchModule:
    """
    Module for performing web searches to fact-check claims using the Google Fact Check API.
    """
    
    def __init__(self, api_key: Optional[str] = None):
        """
        Initialize the WebSearchModule.
        
        Args:
            api_key: API key for the Google Fact Check API
        """
        self.api_key = api_key or os.environ.get("FACT_CHECK_API_KEY")
        if not self.api_key:
            logger.warning("No API key provided. Fact-checking functionality will not work.")
            
        self.fact_check_base_url = "https://factchecktools.googleapis.com/v1alpha1/claims:search"
        self.cache = {}  # Simple in-memory cache
    
    def fact_check_claim(self, claim: str, language_code: str = "en") -> Dict[str, Any]:
        """
        Query the Google Fact Check API for a specific claim.
        
        Args:
            claim: The claim text to fact-check
            language_code: BCP-47 language code (default: "en")
            
        Returns:
            Dictionary containing fact-check results
        """
        # Check if API key is available
        if not self.api_key:
            logger.error("Cannot fact-check claim: No API key provided")
            return {"error": "No API key provided", "claims": []}
        
        # Check cache first
        cache_key = f"{claim}_{language_code}"
        if cache_key in self.cache:
            logger.info(f"Cache hit for claim: {claim[:30]}...")
            return self.cache[cache_key]
        
        # Prepare request parameters
        params = {
            "key": self.api_key,
            "query": claim,
            "languageCode": language_code
        }
        
        try:
            logger.info(f"Querying Google Fact Check API for claim: {claim[:30]}...")
            response = requests.get(self.fact_check_base_url, params=params)
            response.raise_for_status()
            
            result = response.json()
            
            # Cache the result
            self.cache[cache_key] = result
            
            return result
        
        except requests.exceptions.RequestException as e:
            logger.error(f"Error querying Google Fact Check API: {str(e)}")
            return {"error": str(e), "claims": []}
    
    def process_fact_check_results(self, results: Dict[str, Any]) -> Tuple[float, List[Dict[str, Any]]]:
        """
        Process fact-check results to determine credibility score.
        
        Args:
            results: The raw results from the fact_check_claim method
            
        Returns:
            Tuple containing:
            - credibility_score: Float between 0.0 (likely fake) and 1.0 (likely true)
            - evidence: List of dictionaries containing supporting evidence
        """
        if "error" in results:
            logger.warning(f"Cannot process results due to error: {results['error']}")
            return 0.5, []  # Neutral score when API fails
        
        claims = results.get("claims", [])
        if not claims:
            logger.info("No fact-check results found for the claim")
            return 0.5, []  # Neutral score when no results
        
        evidence = []
        rating_sum = 0.0
        rating_count = 0
        
        for claim in claims:
            claim_reviews = claim.get("claimReview", [])
            
            for review in claim_reviews:
                # Extract publisher info
                publisher = review.get("publisher", {}).get("name", "Unknown Source")
                
                # Extract rating
                rating_value = self._extract_rating_value(review)
                if rating_value is not None:
                    rating_sum += rating_value
                    rating_count += 1
                
                # Collect evidence
                evidence.append({
                    "publisher": publisher,
                    "title": review.get("title", ""),
                    "url": review.get("url", ""),
                    "rating": review.get("textualRating", ""),
                    "rating_value": rating_value
                })
        
        # Calculate average credibility score
        credibility_score = rating_sum / rating_count if rating_count > 0 else 0.5
        
        return credibility_score, evidence
    
    def _extract_rating_value(self, review: Dict[str, Any]) -> Optional[float]:
        """
        Extract a normalized rating value from a review.
        
        Args:
            review: A single review from the fact-check results
            
        Returns:
            Float between 0.0 (false) and 1.0 (true), or None if not available
        """
        # Try to get the rating value
        rating = review.get("textualRating", "").lower()
        
        # Map common rating terms to numerical values
        if any(term in rating for term in ["false", "pants on fire", "fake"]):
            return 0.0
        elif any(term in rating for term in ["mostly false", "misleading"]):
            return 0.25
        elif any(term in rating for term in ["mixture", "half true", "partly"]):
            return 0.5
        elif any(term in rating for term in ["mostly true"]):
            return 0.75
        elif any(term in rating for term in ["true", "correct", "accurate"]):
            return 1.0
        
        # If no match, try to extract numerical rating
        try:
            rating_value = float(review.get("ratingValue", 0))
            max_rating = float(review.get("maxRating", 1))
            return rating_value / max_rating
        except (ValueError, TypeError, ZeroDivisionError):
            return None
    
    def check_claim_credibility(self, claim: str) -> Dict[str, Any]:
        """
        High-level method to check the credibility of a claim.
        
        Args:
            claim: The claim to check
            
        Returns:
            Dictionary with credibility assessment and evidence
        """
        # Query the fact-check API
        fact_check_results = self.fact_check_claim(claim)
        
        # Process the results
        credibility_score, evidence = self.process_fact_check_results(fact_check_results)
        
        return {
            "claim": claim,
            "credibility_score": credibility_score,
            "evidence": evidence,
            "has_fact_checks": len(evidence) > 0
        }

## 5. Result Integrator Implementation

This component integrates results from different sources (e.g., CLIP model and fact-checking).

In [ ]:
class ResultIntegrator:
    """
    Module for integrating CLIP model predictions with fact-check results.
    """
    
    def __init__(self, clip_weight: float = 0.6, fact_check_weight: float = 0.4):
        """
        Initialize the ResultIntegrator.
        
        Args:
            clip_weight: Weight given to CLIP model prediction (default: 0.6)
            fact_check_weight: Weight given to fact-check results (default: 0.4)
        """
        self.clip_weight = clip_weight
        self.fact_check_weight = fact_check_weight
        
        # Ensure weights sum to 1.0
        total_weight = self.clip_weight + self.fact_check_weight
        if total_weight != 1.0:
            self.clip_weight /= total_weight
            self.fact_check_weight /= total_weight
            
        logger.info(f"Initialized ResultIntegrator with weights: CLIP={self.clip_weight:.2f}, Fact-check={self.fact_check_weight:.2f}")
    
    def integrate_results(self, 
                         clip_result: Dict[str, Any], 
                         fact_check_results: List[Dict[str, Any]]) -> Dict[str, Any]:
        """
        Integrate CLIP model prediction with fact-check results.
        
        Args:
            clip_result: Dictionary containing CLIP model prediction
                Expected format: {"label": 0 or 1, "confidence": float}
                where 0 = fake, 1 = real
            fact_check_results: List of dictionaries containing fact-check results
                Expected format: [{"claim": str, "credibility_score": float, "evidence": list}]
        
        Returns:
            Dictionary containing integrated results
        """
        # Extract CLIP prediction
        clip_label = clip_result.get("label", 0)
        clip_confidence = clip_result.get("confidence", 0.5)
        
        # Convert CLIP binary label to score (0 = fake = 0.0, 1 = real = 1.0)
        clip_score = float(clip_label)
        
        # Apply confidence to make the score more nuanced
        # If label is 0 (fake), confidence of 0.8 means score of 0.2
        # If label is 1 (real), confidence of 0.8 means score of 0.8
        if clip_label == 0:
            clip_score = 1.0 - clip_confidence
        else:
            clip_score = clip_confidence
            
        logger.info(f"CLIP score: {clip_score:.4f} (label={clip_label}, confidence={clip_confidence:.4f})")
        
        # Process fact-check results
        if not fact_check_results:
            logger.warning("No fact-check results provided, using CLIP prediction only")
            return self._prepare_final_result(clip_score, None, clip_result, [])
        
        # Calculate average credibility score from fact-check results
        total_score = 0.0
        total_weight = 0.0
        all_evidence = []
        
        for result in fact_check_results:
            credibility_score = result.get("credibility_score", 0.5)
            evidence = result.get("evidence", [])
            claim = result.get("claim", "")
            
            # Weight by evidence count (more evidence = more reliable)
            weight = min(len(evidence), 5) / 5.0 if evidence else 0.5
            total_score += credibility_score * weight
            total_weight += weight
            
            # Collect all evidence
            all_evidence.extend(evidence)
            
            logger.info(f"Fact-check for claim '{claim[:30]}...': score={credibility_score:.4f}, weight={weight:.2f}")
        
        # Calculate weighted average of fact-check scores
        fact_check_score = total_score / total_weight if total_weight > 0 else 0.5
        logger.info(f"Overall fact-check score: {fact_check_score:.4f}")
        
        # Integrate CLIP and fact-check scores
        return self._prepare_final_result(clip_score, fact_check_score, clip_result, fact_check_results)
    
    def _prepare_final_result(self, 
                             clip_score: float, 
                             fact_check_score: Optional[float], 
                             clip_result: Dict[str, Any],
                             fact_check_results: List[Dict[str, Any]]) -> Dict[str, Any]:
        """
        Prepare the final integrated result.
        
        Args:
            clip_score: Score from CLIP model (0.0 to 1.0)
            fact_check_score: Score from fact-check results (0.0 to 1.0) or None
            clip_result: Original CLIP result dictionary
            fact_check_results: List of fact-check result dictionaries
            
        Returns:
            Dictionary containing final integrated results
        """
        # If no fact-check score, use only CLIP score
        if fact_check_score is None:
            integrated_score = clip_score
            weights_used = {"clip": 1.0, "fact_check": 0.0}
        else:
            # Weighted average of CLIP and fact-check scores
            integrated_score = (clip_score * self.clip_weight) + (fact_check_score * self.fact_check_weight)
            weights_used = {"clip": self.clip_weight, "fact_check": self.fact_check_weight}
        
        logger.info(f"Integrated score: {integrated_score:.4f}")
        
        # Determine final label and confidence
        final_label = 1 if integrated_score >= 0.5 else 0
        final_confidence = abs(integrated_score - 0.5) * 2  # Scale to 0-1
        
        # Collect evidence from fact-check results
        all_evidence = []
        for result in fact_check_results:
            evidence = result.get("evidence", [])
            claim = result.get("claim", "")
            all_evidence.extend([{**item, "claim": claim} for item in evidence])
        
        # Prepare final result
        final_result = {
            "label": final_label,  # 0 = fake, 1 = real
            "label_text": "real" if final_label == 1 else "fake",
            "confidence": final_confidence,
            "integrated_score": integrated_score,
            "component_scores": {
                "clip": clip_score,
                "fact_check": fact_check_score
            },
            "weights_used": weights_used,
            "evidence": all_evidence,
            "clip_details": clip_result,
            "fact_check_details": fact_check_results
        }
        
        return final_result

## 6. Fake News Detector Implementation

This is the main component that orchestrates the entire process.

In [ ]:
class FakeNewsDetector:
    """
    Main controller for fake news detection with web search integration.
    """
    
    def __init__(self, 
                api_key: Optional[str] = None,
                clip_weight: float = 0.6,
                fact_check_weight: float = 0.4,
                max_claims: int = 3):
        """
        Initialize the FakeNewsDetector.
        
        Args:
            api_key: API key for Google Fact Check API
            clip_weight: Weight given to CLIP model prediction
            fact_check_weight: Weight given to fact-check results
            max_claims: Maximum number of claims to extract and check
        """
        self.api_key = api_key or API_KEY
        self.max_claims = max_claims
        
        # Initialize components
        logger.info("Initializing FakeNewsDetector components")
        self.claim_extractor = ClaimExtractor()
        self.web_search = WebSearchModule(api_key=self.api_key)
        self.result_integrator = ResultIntegrator(clip_weight, fact_check_weight)
        
        # CLIP model is not included in this notebook
        # In a real implementation, you would load it here
        self.clip_model = None
    
    def detect_web_search_only(self, text: str) -> Dict[str, Any]:
        """
        Detect fake news using only web search (without CLIP model).
        
        Args:
            text: The news text to classify
            
        Returns:
            Dictionary containing the detection result
        """
        logger.info(f"Processing text with web search only (length {len(text)} characters)")
        
        # Extract claims from text
        claims = self.claim_extractor.extract_claims(text)
        logger.info(f"Extracted {len(claims)} claims from text")
        
        # Rank and select top claims
        top_claims = self.claim_extractor.rank_claims(claims, top_n=self.max_claims)
        logger.info(f"Selected top {len(top_claims)} claims for fact-checking")
        
        # Check claims with web search
        fact_check_results = []
        for claim in top_claims:
            result = self.web_search.check_claim_credibility(claim)
            fact_check_results.append(result)
            logger.info(f"Fact-check for claim '{claim[:30]}...': score={result['credibility_score']:.4f}")
        
        # Calculate average credibility score
        if fact_check_results:
            total_score = sum(r['credibility_score'] for r in fact_check_results)
            avg_score = total_score / len(fact_check_results)
        else:
            avg_score = 0.5  # Neutral score if no results
        
        # Determine final label and confidence
        final_label = 1 if avg_score >= 0.5 else 0
        final_confidence = abs(avg_score - 0.5) * 2  # Scale to 0-1
        
        # Collect all evidence
        all_evidence = []
        for result in fact_check_results:
            evidence = result.get("evidence", [])
            claim = result.get("claim", "")
            all_evidence.extend([{**item, "claim": claim} for item in evidence])
        
        return {
            "label": final_label,  # 0 = fake, 1 = real
            "label_text": "real" if final_label == 1 else "fake",
            "confidence": final_confidence,
            "credibility_score": avg_score,
            "claims_checked": top_claims,
            "fact_check_results": fact_check_results,
            "evidence": all_evidence
        }
    
    def detect(self, text: str, image_path: Optional[str] = None) -> Dict[str, Any]:
        """
        Detect fake news using both CLIP model and web search.
        Note: This method is a placeholder since we don't have the CLIP model in this notebook.
        
        Args:
            text: The news text to classify
            image_path: Path to the associated image (optional)
            
        Returns:
            Dictionary containing the detection result
        """
        logger.warning("CLIP model is not available in this notebook. Using web search only.")
        
        # In a real implementation, you would:
        # 1. Make prediction with CLIP model
        # 2. Extract and check claims with web search
        # 3. Integrate results
        
        # For now, just use web search
        web_search_result = self.detect_web_search_only(text)
        
        # Create a mock CLIP result
        mock_clip_result = {
            "label": web_search_result["label"],  # Use the same label for demonstration
            "confidence": 0.7  # Arbitrary confidence value
        }
        
        # Integrate the results (this will be weighted heavily toward the web search result)
        return self.result_integrator.integrate_results(
            mock_clip_result, 
            web_search_result["fact_check_results"]
        )

## 7. Testing the Implementation

Let's test the fake news detection system with some example news texts.

In [ ]:
# Initialize the detector with your API key
detector = FakeNewsDetector(api_key=API_KEY)

# Test cases
test_cases = [
    {
        "title": "Likely Fake News (COVID-19 microchips)",
        "text": """
        Breaking News: Scientists have discovered that COVID-19 vaccines contain microchips 
        that allow governments to track individuals. According to Dr. John Smith, these 
        microchips are activated by 5G networks and can control people's thoughts. 
        The World Health Organization has been hiding this information from the public.
        """
    },
    {
        "title": "Likely Real News (Health benefits)",
        "text": """
        A new study published in the Journal of Medicine found that regular exercise 
        can reduce the risk of heart disease by up to 30%. The research, conducted over 
        a 10-year period with 5,000 participants, shows a clear correlation between 
        physical activity and cardiovascular health. Health experts recommend at least 
        150 minutes of moderate exercise per week.
        """
    },
    {
        "title": "Conspiracy Theory (Flat Earth)",
        "text": """
        New evidence suggests that the Earth is actually flat, contrary to what scientists 
        have been telling us for centuries. Photos from space are manipulated by NASA to 
        maintain the illusion of a spherical Earth. The truth is being hidden from the public 
        to maintain control over the population.
        """
    }
]

# Process each test case
for i, test_case in enumerate(test_cases, 1):
    print("\n" + "="*80)
    print(f"Test Case {i}: {test_case['title']}")
    print("-"*80)
    print(f"Text: {test_case['text'].strip()}")
    print("-"*80)
    
    # Detect fake news using web search only
    result = detector.detect_web_search_only(test_case["text"])
    
    # Print the result
    print(f"Verdict: {result['label_text'].upper()}")
    print(f"Confidence: {result['confidence']:.2f}")
    
    # Print claims checked
    if result['claims_checked']:
        print("\nClaims checked:")
        for j, claim in enumerate(result['claims_checked'], 1):
            print(f"{j}. {claim}")
    
    # Print evidence
    if result['evidence']:
        print("\nEvidence:")
        for j, item in enumerate(result['evidence'], 1):
            print(f"{j}. {item['publisher']}: {item['rating']}")
            print(f"   URL: {item['url']}")
    else:
        print("\nNo fact-check evidence found.")

## 8. Try Your Own Text

Now you can try the system with your own text:

In [ ]:
# Get text from user
user_text = input("Enter your text to check for fake news: ")

if user_text:
    print("\n" + "="*80)
    print("Processing your text...")
    print("-"*80)
    
    # Detect fake news using web search only
    result = detector.detect_web_search_only(user_text)
    
    # Print the result
    print(f"Verdict: {result['label_text'].upper()}")
    print(f"Confidence: {result['confidence']:.2f}")
    
    # Print claims checked
    if result['claims_checked']:
        print("\nClaims checked:")
        for j, claim in enumerate(result['claims_checked'], 1):
            print(f"{j}. {claim}")
    
    # Print evidence
    if result['evidence']:
        print("\nEvidence:")
        for j, item in enumerate(result['evidence'], 1):
            print(f"{j}. {item['publisher']}: {item['rating']}")
            print(f"   URL: {item['url']}")
    else:
        print("\nNo fact-check evidence found.")
else:
    print("No text entered.")

## 9. Conclusion

This notebook has demonstrated a complete implementation of a fake news detection system with web search integration using the Google Fact Check API. The system:

1. Extracts claims from news text using NLP techniques
2. Checks these claims against the Google Fact Check API
3. Determines the credibility of the news based on fact-check results

For a full implementation, you would integrate this with a CLIP-based model for multimodal analysis, which would improve the accuracy of the system by considering both textual and visual content.

### Next Steps

1. Integrate with a real CLIP model for multimodal analysis
2. Improve the claim extraction with more sophisticated NLP techniques
3. Add support for multiple languages
4. Implement a caching system for faster processing
5. Add support for other fact-checking APIs